In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (ConvLSTM2D, BatchNormalization, Convolution3D, Convolution2D,
                          TimeDistributed, MaxPooling2D, UpSampling2D, Input, concatenate)

In [12]:
c = 12
input_shape=(4,48,48,3)
input_img = Input(input_shape, name='input')
x = ConvLSTM2D(c, (3,3), padding='same', return_sequences=True)(input_img)
x = ConvLSTM2D(c, (3,3), padding='same', return_sequences=True)(x)
c1 = ConvLSTM2D(c, (3,3), padding='same', return_sequences=True)(x)

x = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(c1)

x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
c2 = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x) # 

x = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(c2)
x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
c3 = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)

x = TimeDistributed(UpSampling2D((2, 2)))(c3)
x = concatenate([c2, x])
x = TimeDistributed(Convolution2D(c, 3, 3, padding='same'))(x)

x = TimeDistributed(UpSampling2D((2, 2)))(x)
    
model = Model(inputs=input_img, outputs=x)
print(model.summary())

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 4, 48, 48, 3 0                                            
__________________________________________________________________________________________________
conv_lst_m2d_66 (ConvLSTM2D)    (None, 4, 48, 48, 12 6528        input[0][0]                      
__________________________________________________________________________________________________
conv_lst_m2d_67 (ConvLSTM2D)    (None, 4, 48, 48, 12 10416       conv_lst_m2d_66[0][0]            
__________________________________________________________________________________________________
conv_lst_m2d_68 (ConvLSTM2D)    (None, 4, 48, 48, 12 10416       conv_lst_m2d_67[0][0]            
____________________________________________________________________________________________

In [17]:
def class_net_fcn_2p_lstm(input_shape):
    #concat_axis = 4
    c = 12
    input_img = Input(input_shape, name='input')
    x = ConvLSTM2D(c, (3,3), padding='same', return_sequences=True)(input_img)
    x = ConvLSTM2D(c, (3,3), padding='same', return_sequences=True)(x)
    c1 = ConvLSTM2D(c, (3,3), padding='same', return_sequences=True)(x) #48x48

    x = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(c1)

    x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
    x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
    c2 = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x) #24x24

    x = TimeDistributed(MaxPooling2D((2, 2), (2, 2)))(c2)
    x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
    x = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x)
    c3 = ConvLSTM2D(2 * c, (3,3), padding='same', return_sequences=True)(x) #12x12

    x = TimeDistributed(UpSampling2D((2, 2)))(c3)
    x = concatenate([c2, x])
    x = TimeDistributed(Convolution2D(c, 3, 3, padding='same'))(x)

    x = TimeDistributed(UpSampling2D((2, 2)))(x)
    x = concatenate([c1, x])
    # x = TimeDistributed(Convolution2D(c, 3, 3, border_mode='same'))(x)

    x = TimeDistributed(Convolution2D(3, 3, 3, padding='same'))(x)
    x = TimeDistributed(UpSampling2D((2, 2)))(x)
    x = TimeDistributed(Convolution2D(3, 3, 3, padding='same'))(x)
    x = TimeDistributed(UpSampling2D((2, 2)))(x)

    output = TimeDistributed(Convolution2D(3, 3, 3, padding='same', activation=softmax_2d(-1)), name='output')(x)

    model = Model(input_img, output=[output])
    model.compile(loss=categorical_crossentropy_3d_w(2, class_dim=-1), optimizer='adadelta')
    print(model.summary())
    return model

In [18]:
model = class_net_fcn_2p_lstm((4,48,48,3))

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 4, 48, 48, 12), (None, 4, 16, 16, 12)]